In [21]:
import json

# Open the dataset JSON
with open("dataSets/ingredient_and_instructions.json", "r") as dataSet:
    data = json.load(dataSet)

recipie = "" # The individual recipie
recipie_list = [] # Where we will store the recipie for training

for meal in data:
    # Get the recipie name and its instructions
    meal_info = data[meal]
    recipie += meal.replace("-", " ").title() + "\n\n"
    recipe_instructions = meal_info["instructions"]

    # Get the ingredients
    ingredients = meal_info["ingredient_sections"]

    for i in ingredients:
        # Retrieve ingredients and their details
        item = ""           # Dough
        qty = ""            # ½
        unit = ""           # cup
        ingredient = ""     # unsalted butter
        primary_unit = { "quantity": None, "display": None }

        # Some ingredients arent named, but are still listed as "1 cup" or similar
        # Additionally, if there is an ingredient name, it may already have a colon
        # in front of it, so we need to check for that before formatting
        if i["name"]:
            if i["name"][-1] != ":": # If there isnt a colon, add one
                item = i["name"] + ": \n"
            else: # Otherwise, just add the name
                item = i["name"] + " \n"
        else:
            item = ""

        recipie += item # Add the ingredient name to the recipie

        # Loop over each ingredient
        j = 0
        while j < len(i["ingredients"]):
            # i["ingredients"][0] may not exist, use try/except to prevent
            # any ListIndexErrors from accessing an index that doesn't exist
            try: primary_unit = i["ingredients"][j]["primary_unit"]
            except: primary_unit = { "quantity": None, "display": None } # Dummy object

            # Some recipies don't list an ingredient at all
            try: ingredient += i["ingredients"][j]["name"]
            except: ingredient += ""

            # If the try/except blocks above are successful, we can set the values
            if primary_unit["quantity"]: qty = primary_unit["quantity"] + " "
            if primary_unit["display"]: unit = primary_unit["display"] + " "

            # Construct the ingredient string
            recipie += qty + unit  + ingredient + "\n"
            ingredient = ""
            j += 1

        recipie += "\n"

    # Newline before adding instructions
    recipie += "\n"

    # Loop over the instructions and format them
    step = 1
    for text in recipe_instructions:
        instruction = text["display_text"]
        recipie += str(step) + ". " + instruction + "\n"
        step += 1

    # Add the recipie to the list
    recipie_list.append(recipie)

    # Free memory for the next recipie
    recipie = ""
    recipe_instructions = []
    ingredients = []
    item = ""
    qty = ""
    unit = ""
    ingredient = ""

# For testing purposes, we can output a random recipie
import random
print(recipie_list[random.randint(0, len(recipie_list) - 1)])

#print(recipie_list[0])


Sweet Potato Breakfast Cups

3 medium sweet potatoes
1 teaspoon salt
½ teaspoon pepper
½ teaspoon paprika
1 teaspoon garlic powder
1 teaspoon onion powder
2 tablespoons olive oil
1 cup shredded cheese blend

Filling Suggestions: 

Egg And Pepper: 
2 small eggs
¼ red bell pepper

Broccoli, Tomato, And Cheese: 
½ cup small broccoli floret
½ cup cherry tomato
¼ cup shredded cheese blend

Pepper, Onion, And Bacon: 
¼ red bell pepper
¼ cup onion
1 ½ strips bacon

Egg And Bacon: 
3 small eggs
1 ½ strips bacon


1. Preheat the oven to 400°F (200°C).
2. With a mandolin or a sharp knife, carefully cut the sweet potatoes into ¹⁄₁₆-inch (1 mm) thick slices.
3. Place the sweet potato slices in a large bowl, add the salt, pepper, paprika, garlic powder, and onion powder, and toss the potatoes until evenly coated.
4. Let sit for about 20 minutes, until water releases from the sweet potatoes and the slices are soft and flexible.
5. Brush the cups of a 12-cup muffin tin with the olive oil.
6. Arrange 

In [22]:
#CSV File 1 Record per row
#Record: 1 string with ingredients + recipe


# [[recipe1], [recipe2], [recipe3]]

import csv

with open("recipe.csv", 'w', encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    for recipie in recipie_list:
        csvwriter.writerow([recipie])


In [41]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

file_name = "recipe.csv"

train_tokenizer(file_name)
tokenizer_file = "aitextgen.tokenizer.json"
config = GPT2ConfigCPU()
ai = aitextgen(tokenizer_file=tokenizer_file, config=config)
data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)
ai.train(data, batch_size=32, num_steps=120000, generate_every=1000, save_every=5000)
ai.generate(prompt="steak")

You are tokenizing a CSV file, but you did not set line_by_line=True. Please change if unintended.


In [40]:
ai.generate(prompt="cheese cake")

cheese cake with an offb and any large spatula.
11. Remove the cakes from the oven. In a large mixing bowl, combine the eggs, salt, baking soda, rainbow spice, and cinnamon. Set aside.
12. Heat
